In [ ]:
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import pywt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seem for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
# Root directory for dataset
dataroot = "/hdd_c/sri/CelebA/512/"

# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 256

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 128

# Size of feature maps in discriminator
ndf = 64

# Size of conditioning label
ncl = 1

# Number of training epochs
num_epochs = 10

# Learning rate for optimizers
lr_D = 0.0002
lr_G = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1


In [ ]:
class CenterCropLongEdge(object):
  """Crops the given PIL Image on the long edge.
  Args:
      size (sequence or int): Desired output size of the crop. If size is an
          int instead of sequence like (h, w), a square crop (size, size) is
          made.
  """
  def __call__(self, img):
    """
    Args:
        img (PIL Image): Image to be cropped.
    Returns:
        PIL Image: Cropped image.
    """
    return transforms.functional.center_crop(img, min(img.size))

  def __repr__(self):
    return self.__class__.__name__

In [ ]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               CenterCropLongEdge(),
                               transforms.Resize(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers, drop_last=True)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))

# print(np.min(real_batch[0][0].numpy()))

plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:24], padding=2, normalize=False).cpu(),(1,2,0)))


In [ ]:
def wt(vimg, filters, levels=1):
    bs = vimg.shape[0]
    h = vimg.size(2)
    w = vimg.size(3)
    vimg = vimg.reshape(-1, 1, h, w)
    padded = torch.nn.functional.pad(vimg,(2,2,2,2))
    res = torch.nn.functional.conv2d(padded, Variable(filters[:,None]),stride=2)
    if levels>1:
        res[:,:1] = wt(res[:,:1], filters, levels-1)
        res[:,:1,32:,:] = res[:,:1,32:,:]*1.
        res[:,:1,:,32:] = res[:,:1,:,32:]*1.
        res[:,1:] = res[:,1:]*1.
    res = res.view(-1,2,h//2,w//2).transpose(1,2).contiguous().view(-1,1,h,w)
    return res.reshape(bs, -1, h, w)

def create_filters(device, wt_fn='bior2.2'):
    w = pywt.Wavelet(wt_fn)
    dec_hi = torch.Tensor(w.dec_hi[::-1]).to(device)
    dec_lo = torch.Tensor(w.dec_lo[::-1]).to(device)
    filters = torch.stack([dec_lo.unsqueeze(0)*dec_lo.unsqueeze(1),
                           dec_lo.unsqueeze(0)*dec_hi.unsqueeze(1),
                           dec_hi.unsqueeze(0)*dec_lo.unsqueeze(1),
                           dec_hi.unsqueeze(0)*dec_hi.unsqueeze(1)], dim=0)
    return filters

filters = create_filters(device='cuda:0')

cur_max = float('-inf')
cur_min = float('inf')
for i, data in enumerate(dataloader):
    data_512 = data[0].to('cuda:0')
    data_wt = wt(data_512, filters=filters, levels=3)[:, :, :32, :32]
    cur_max = max(cur_max, torch.max(data_wt))
    cur_min = min(cur_min, torch.min(data_wt))
print(cur_max)
print(cur_min)

In [ ]:
shift = -1*cur_min
scale = shift+cur_max

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
        
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
# Variational Autoencoder Code
'''Sub class the nn module'''
class AE(nn.Module):
    def __init__(self, ngpu):
        super(AE, self).__init__() #not a python thing
        self.ngpu = ngpu
        
        self.encoder = nn.Sequential(
            nn.Conv2d(nc+ncl, ndf//2, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # input is (nc) x 16 x 16
            nn.Conv2d(ndf//2, ndf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.ReLU(True),
            # state size. (ndf) x 8 x 8
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            # state size. (ndf*2) x 4 x 4
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            # state size. (ndf*4) x 2 x 2
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True),
            # state size. (ndf*8) x 1 x 1
            Flatten()
        )
        
        self.fc1 = nn.Linear(ndf*8*1*1, nz)
        self.fc2 = nn.Linear(ndf*8*1*1, nz)
        self.fc3 = nn.Linear(nz+ncl, ndf*8*1*1)
        
        self.decoder = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( ndf*8*1*1, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*8) x 2 x 2
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*4) x 4 x 4
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf*2) x 8 x 8
            nn.ConvTranspose2d( ngf, ngf//2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf//2),
            nn.ReLU(True),
            # state size. (ngf) x 16 x 16
            nn.ConvTranspose2d( ngf//2, nc, 4, 2, 1, bias=False),
            nn.Sigmoid()
            # state size. (nc) x 32 x 32
        )
        
    def reparameterize(self, mu, logvar):
        eps = torch.randn(*mu.size(),device=device)
        std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        z = mu + (std*eps)
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = z.unsqueeze(-1).unsqueeze(-1)
        x = self.decoder(z)
        return x

    def forward(self, x, c):
        c_encoder = c.expand(x.shape[0], ncl, x.shape[2], x.shape[3])
        x = torch.cat([x, c_encoder], dim=1)
        z, mu, logvar = self.encode(x)
        
        c_decoder = c
        z = torch.cat([z, c_decoder], dim=1)
        x = self.decode(z)
        return x, mu, logvar


# Create the generator
netAE = AE(ngpu) #.to(device) # adding it to the device

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netAE = nn.DataParallel(netAE)
    
netAE.to(device)

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netAE.apply(weights_init)

# Print the model
print(netAE)
print('Active CUDA Device: GPU', torch.cuda.current_device())
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

`Input: (N,Cin,Hin,Win)
Output: (N,Cout,Hout,Wout) where Hout=(Hin−1)∗stride[0]−2∗padding[0]+kernel_size[0]+output_padding[0]  Wout=(Win−1)∗stride[1]−2∗padding[1]+kernel_size[1]+output_padding[1]`

In [ ]:
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD, BCE, KLD

# Setup Adam optimizers for both G and D
optimizerAE = optim.Adam(netAE.parameters(), lr=lr_G, betas=(beta1, 0.999))

In [ ]:
netAE.load_state_dict(checkpoint['netAE_state_dict'])
optimizerAE.load_state_dict(checkpoint['optimizer_netAE_state_dict'])

In [ ]:
# Training Loop
netAE.train()

# Lists to keep track of progress
AE_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        
        ############################
        # (1) Update AE network
        ###########################
        ## Train with all-real batch
        netAE.zero_grad() 
        
        img = data[0].to(device) 
        
        img = wt(img, filters=filters, levels=4)[:, :, :32, :32]
        img = (img+shift)/scale
        b_size = img.size(0)
        recon_images, mu, logvar = netAE(img)
        errAE, bce, kld = loss_fn(recon_images, img, mu, logvar)
        optimizerAE.zero_grad()
        errAE.backward()
        optimizerAE.step()
        
        
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_VAE: %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errAE.item()))
            
        iters += 1

In [ ]:
with torch.no_grad():
    netAE.eval()
    data = next(iter(dataloader))
    img = data[0][:32].to(device)
    img = wt(img, filters=filters, levels=4)[:, :, :32, :32]
    grid = (img+shift)/scale
#     plt.imshow(np.transpose(grid[0].cpu(),(1,2,0)))
    grid,_,_ = netAE(grid)
    grid = grid.detach().cpu()
    plt.imshow(np.transpose(vutils.make_grid(grid, padding=2, normalize=False).cpu(),(1,2,0)))
        

In [ ]:
# Decide which device we want to run on
device_2 = torch.device("cuda:1" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# Create a copy of filters on the second device
filters_d2 = create_filters(device=device_2)

# Generator Code
'''Sub class the nn module'''
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__() #not a python thing
        self.ngpu = ngpu
        self.encoder = nn.Sequential(
            nn.Conv2d(nc, ndf//2, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # input is (nc) x 16 x 16
            nn.Conv2d(ndf//2, ndf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.ReLU(True),
            # state size. (ndf) x 8 x 8
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            # state size. (ndf*2) x 4 x 4
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            # state size. (ndf*4) x 2 x 2
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True),
            # state size. (ndf*8) x 1 x 1
            Flatten()
        )
        
        self.decoder = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( ndf*8*1*1, ngf * 4, 4, 2, 0, bias=False),
            nn.InstanceNorm2d(ngf * 4,affine=True),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ngf * 2,affine=True),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ngf,affine=True),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
#             nn.InstanceNorm2d(ngf//2,affine=True),
#             nn.ReLU(True),
#             # state size. (ngf) x 32 x 32
#             nn.ConvTranspose2d( ngf//2, nc, 4, 2, 1, bias=False),
            nn.Sigmoid()
            # state size. (nc) x 32 x 32
        )

    def forward(self, input):
#         eps = torch.randn(b_size, 3, 64, 64, device=device)
        enc = self.encoder(input)
        enc = enc.unsqueeze(-1).unsqueeze(-1)
        
        return self.decoder(enc)

# Create the generator
netG = Generator(ngpu).to(device_2) # adding it to the device

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

In [ ]:
def plot():
    with torch.no_grad():
        netAE.eval()
        data = next(iter(dataloader))
        img = data[0][:32].to(device)
        img = wt(img, filters=filters, levels=4)[:, :, :32, :32]
        img = (img+shift)/scale
        b_size = img.size(0)
        grid,_,_ = netAE(img)
        grid = grid.detach().cpu()
        plt.figure(figsize=(8,8))
        plt.axis("off")
        plt.title("AE Reconstruction")
        plt.imshow(np.transpose(vutils.make_grid(grid, padding=2, normalize=False).cpu(),(1,2,0)))
        
        netG.eval()
        grid = netG(grid.to(device_2)).detach().cpu()
        plt.figure(figsize=(8,8))
        plt.axis("off")
        plt.title("GAN Reconstruction")
        plt.imshow(np.transpose(vutils.make_grid(grid, padding=2, normalize=False).cpu(),(1,2,0)))
        plt.show()

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(batch_size*2, nz, 1, 1, device=device) #adding it to the device

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

# Setup Adam optimizers for both G and D
optimizerG = optim.Adam(netG.parameters(), lr=lr_G, betas=(beta1, 0.999))

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(30*num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        
        #Put data on cuda:0
        real_cpu = data[0].to(device) 
        real_cpu = wt(real_cpu, filters=filters, levels=4)[:, :, :32, :32]
        real_cpu = (real_cpu+shift)/scale
        
        with torch.no_grad():
            netAE.eval()
            class_,_,_ = netAE(real_cpu)
            class_ = class_.detach().to(device_2)
        
        
        #Put data on cuda:1
        real_cpu = data[0].to(device_2)
        real_cpu = wt(real_cpu, filters=filters_d2, levels=4)[:, :, :32, :32]
        real_cpu = (real_cpu+shift)/scale


        ## Generate
        noise = torch.randn(b_size, nz, 1, 1, device=device_2)
        fake = netG(class_)

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        errG = criterion(fake,real_cpu)
        errG.backward()
        optimizerG.step()
        
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_G: %.4f'
                  % (epoch, num_epochs, i, len(dataloader),errG.item()))
        if i % 100 == 0:
            plot()
            netG.train()
        
            
        iters += 1

In [ ]:
torch.save({
            'netAE_state_dict': netAE.state_dict(),
            'netG_state_dict': netG.state_dict(),
            'optimizer_netAE_state_dict': optimizerAE.state_dict(),
            'optimizer_netG_state_dict': optimizerG.state_dict()
            }, 'DSVAE_WT_32.pt')

import gc
gc.collect()